# Building Data Genome - Geographical Regions



In [ ]:
#Installations


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached pyspellchecker-0.8.1-py3-none-any.whl.metadata (9.4 kB)
Using cached pyspellchecker-0.8.1-py3-none-any.whl (6.8 MB)


In [8]:
# Import required packages
import pandas as pd
import numpy as np
from google.colab import drive


pd.set_option('display.max_colwidth', None)

In [15]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pwd

/content


In [25]:
!ls '/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/'

chilledwater_cleaned.csv  hotwater_cleaned.csv	  steam_cleaned.csv
electricity_cleaned.csv   irrigation_cleaned.csv  water_cleaned.csv
gas_cleaned.csv		  solar_cleaned.csv	  weather.csv


In [29]:
#Import csv data into dataframes
df_solar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/solar_cleaned.csv')
df_water = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/water_cleaned.csv')
df_electricity = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/electricity_cleaned.csv')
df_gas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/gas_cleaned.csv')
df_hotwater = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/hotwater_cleaned.csv')
df_irrigation = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/irrigation_cleaned.csv')
df_solar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/solar_cleaned.csv')
df_steam = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/steam_cleaned.csv')

df_metadata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/metadata.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Energy Systems/data/metadata.csv'